In [55]:
import os
import sys
import dspy
import rich
import polars as pl
from pathlib import Path
from typing import List
import pickle

## Load the TCGA Cancer Types

In [42]:
df_cancer_types = pl.read_csv("tcga_abbreviations.csv")
df_cancer_types

abbrev,cancer_type_full
str,str
"""LAML""","""Acute Myeloid Leukemia"""
"""ACC""","""Adrenocortical carcinoma"""
"""BLCA""","""Bladder Urothelial Carcinoma"""
"""LGG""","""Brain Lower Grade Glioma"""
"""BRCA""","""Breast invasive carcinoma"""
…,…
"""THYM""","""Thymoma"""
"""THCA""","""Thyroid carcinoma"""
"""UCS""","""Uterine Carcinosarcoma"""


## Load the Full Pathology Report Dataset

In [43]:
df_path_reports = pl.read_csv("../../data/tcga/TCGA_reports.csv")
assert len(df_path_reports) == 9523
df_path_reports.head(10)

patient_filename,text
str,str
"""TCGA-BP-5195.25c0b433-5557-416…","""Date of Recelpt: Clinical Diag…"
"""TCGA-D7-8573.b7306a47-697d-4ed…","""Material: 1) Material: stomach…"
"""TCGA-EI-7004.13591eed-30e5-47a…","""page 1 / 1. copy No. 3. Examin…"
"""TCGA-EB-A82B.23E186C6-739C-4EF…","""Patient ID: Gross Description:…"
"""TCGA-A6-3808.e1505f65-72ef-438…","""SPECIMEN. Right colon. CLINICA…"
"""TCGA-29-1761.63defa27-e729-451…","""CLINICAL HISTORY: Pelvic organ…"
"""TCGA-24-1616.d8c764d9-0fe8-488…","""Address: : Physician (s) : Oth…"
"""TCGA-IN-A6RO.C0E5F045-3F12-45C…","""FINAL DIAGNOSIS: PART 1: ESOPH…"
"""TCGA-44-8119.c40ea4c8-8cbe-4a4…","""Final Surgical Pathology Repor…"


In [44]:
# print sample text report
rich.print(df_path_reports.select("text").to_series().to_list()[0])

Date of Recelpt: Clinical Diagnosis & History: Incidental 3 cm left upper pole renal mass. Specimens Submitted: 1: 
Kidney, Left Upper Pole; Partial Nephrectomy. DIAGNOSIS: 1. Kidney, Left Upper Pole; Partial Nephrectomy: Tumor 
Type: Renal cell carcinoma - Conventional (clear cell) type. Fuhrman Nuclear Grade: Nuclear grade II/IV. Tumor 
Size: Greatest diameter is 2.4 cm. Local Invasion (for renal cortical types): Not Identified. Renal Vein Invasion: 
Not identified. Surgical Margins: Free of tumor. Non-Neoplastic Kidney: shows focal chronic inflammation and focal 
superficial glomerulosclerosis. Adrenal Gland: Not identified. Lymph Nodes: Not identified. Staging for renal cell 
carcinoma/oncocytoma: pT1 Tumor <= 7.0 cm in greatest dimension limited to the kidney. JATTEST THAT THE ABOVE 
DIAGNOSIS IS BASED UPON MY PERSONAL EXAMINATION OF THE SLIDES (AND/OR OTHER MATERIAL), AND THAT IHAVE. REVIEWED AND
APPROVED THIS REPORT. Gross Description: 1). The specimen Is received fresh for frozen section consultation, 
labeled "left upper pole renal tumor stitch marks deep margin". It consists of a 6.6 X 3.7 x 2.9 cm wedge shaped 
portion of kidney with a suture marking the deep margin and attached 12.2 x 8,5 x. 2.7 cm of perirenal. Also 
received in the same container Is a separate 11.6 x 8.4 x 4.6 cm aggregate of unremarkable. fibroadipose tissue. 
The margin is inked black and the specimen is serially sectioned to reveal a well-circumscribed cortically. based 
yellow focally hemorrhagic 2.4 x 2.0 x 1.9 cm tumor. The tumor appears grossly confined to the renal parenchyma, 
no. extension into perirenal fat is identified. The clearance from the resection margin is 1.1 cm. A representative
section of the. nearest margin is submilted for frozen section diagnosis. Representatively submitted. A photograph 
Is taken. Portions of the. tumor are submitted for TPS. Summary of sections: FSC - frozen section control. T tumor.
M margin. RS - representative sections. Summary of Sections: Part 1: Kidney, Left Upper Pole; Partial Nephrectomy. 
Block. Sect. Site. PCs. 1. fsc. 1. 2. m. 2. 1. rs. 1. 2. t. 2. Intraoperative Consultation: Note: The diagnoses 
given in this section pertain only to the. tissue sample examined at the time of the intraoperative. consultation. 
1). FROZEN SECTION DIAGNOSIS: LEFT UPPER POLE RENAL TUMOR. RENAL CORTICAL NEOPLASM. WITH BENIGN MARGIN. PERMANENT 
DIAGNOSIS: SAME.

## Load the Patient - Cancer Type Dataset

In [45]:
df_patient_type = pl.read_csv(
    "../../tcga-path-reports/data/tcga_metadata/tcga_patient_to_cancer_type.csv"
)
print(df_patient_type.shape)
df_patient_type.head(10)

(11160, 2)


patient_id,cancer_type
str,str
"""TCGA-OR-A5J1""","""ACC"""
"""TCGA-OR-A5J2""","""ACC"""
"""TCGA-OR-A5J3""","""ACC"""
"""TCGA-OR-A5J4""","""ACC"""
"""TCGA-OR-A5J5""","""ACC"""
"""TCGA-OR-A5J6""","""ACC"""
"""TCGA-OR-A5J7""","""ACC"""
"""TCGA-OR-A5J8""","""ACC"""
"""TCGA-OR-A5J9""","""ACC"""


## Final Dataframe: patient_id, path_text, and cancer_type

In [46]:
df_split = df_path_reports.with_columns(
    pl.col("patient_filename")
    .str.split_exact(".", 1)
    .struct.rename_fields(["patient_id", "filename"])
    .alias("fields")
).unnest("fields")
df_split = df_split.drop(["patient_filename", "filename"])
print(df_split.shape)
df_split.head()

(9523, 2)


text,patient_id
str,str
"""Date of Recelpt: Clinical Diag…","""TCGA-BP-5195"""
"""Material: 1) Material: stomach…","""TCGA-D7-8573"""
"""page 1 / 1. copy No. 3. Examin…","""TCGA-EI-7004"""
"""Patient ID: Gross Description:…","""TCGA-EB-A82B"""
"""SPECIMEN. Right colon. CLINICA…","""TCGA-A6-3808"""


In [48]:
df_joined = df_split.join(df_patient_type, on="patient_id", how="inner")
print(df_joined.shape)

df_joined = df_joined.join(
    df_cancer_types, left_on="cancer_type", right_on="abbrev", how="inner"
)
print(df_joined.shape)
df_joined.head()

(9523, 3)
(9523, 4)


text,patient_id,cancer_type,cancer_type_full
str,str,str,str
"""Deliver To: NOS P. Cuncertain …","""TCGA-OR-A5J1""","""ACC""","""Adrenocortical carcinoma"""
"""Date of Birth: Sex: Female. Ma…","""TCGA-OR-A5J2""","""ACC""","""Adrenocortical carcinoma"""
"""Sex: Female. Macroscopy. One v…","""TCGA-OR-A5J3""","""ACC""","""Adrenocortical carcinoma"""
"""Sex: Female. Macroscopy. One v…","""TCGA-OR-A5J4""","""ACC""","""Adrenocortical carcinoma"""
"""Sex: Male. Macroscopy. One ves…","""TCGA-OR-A5J5""","""ACC""","""Adrenocortical carcinoma"""


In [49]:
df_dict = df_joined.to_dicts()
df_dict[0]

{'text': 'Deliver To: NOS P. Cuncertain inknown behavior 8370/1 A. SPECIMEN TYPE: Adrenal. Blopsy No: T. CLINICAL NOTES: H. Histopathology of L) adrenal gland. Conn\'s syndrome (note small section of. O. tumour excised for genetic studies). L. MACROSCOPIC: o. "Left adrenal. The specimen consists of an adrenal gland with a post fixation. weight 102.4g. It measures 110 x 70 x 60mm. A section of tissue has been. G. taken from one pole. On sectioning, a nodular lesion with a maximum diameter. Y. of 25 x 30mm, and extending for a length of 70mm, can be identified. The cut. surface of this lesion is irregular. it can be seen to be compressing the. adjacent normal adrenal tissue. Block 1: tissue near the excision where previous sampling for genetic studies. has already been performed. Blocks 2 and 3: lesion where it abuts the normal adrenal gland. Block 4: lesion with its closest proximity to the excision margin. Blocks 5-9: further blocks of lesion. MICROSCOPIC: Sections show an adrenal tumo

## Create a List of DSPy Examples

In [50]:
def prepare_dataset(dataset) -> List[dspy.Example]:
    return [
        dspy.Example(
            text=row["text"],
            cancer_type=row['cancer_type_full']
        ).with_inputs("text")
        for row in dataset
    ]

In [51]:
examples = prepare_dataset(df_dict)

In [54]:
example = examples[1000]
print(example['cancer_type'])
rich.print(example['text'])

Breast invasive carcinoma


FINAL DIAGNOSIS: PART 1: RIGHT SENTINEL LYMPH NODE #1, BIOPSY -. ONE LYMPH NODE NEGATIVE FOR TUMOR (0/1). PART 2: 
RIGHT BREAST, SEGMENTAL MASTECTOMY -. A. INFILTRATING DUCT CARCINOMA, 1.5 X 1.4 X 1.4 CM. B. NOTTINGHAM SCORE 9/9 
(TUBULES 3, NUCLEAR 3, MITOSES 3). C. NO LYMPHOVASCULAR INVASION IDENTIFIED. D. DUCTAL CARCINOMA IN-SITU SOLID 
TYPE, NUCLEAR GRADE3, WITH COMEDO-TYPE NECROSIS. TISSUE. COMPRISING APPROXIMATELY 3% OF INVASIVE TUMOR AND PRESENT 
IN THE SURROUNDING BREAST. E. SURGICAL MARGINS NEGATIVE FOR INFILTRATING DUCT CARCINOMA AND DUCTAL CARCINOMA IN-. 
SITU. F. G. INFILTRATING DUCT CARCINOMA FOCALLY VERY CLOSELY APPROACHES SUPERIOR MARGIN (0.03 CM). UNREMARKABLE 
SKIN. H. TUMOR IS ESTROGEN RECEPTORS, PROGESTERONE RECEPTORS AND HER2/NEU NEGATIVE AS PER. CORE BIOPSY REPORT. I. 
THERMAL EFFECT. COMMENT: The tumor has pushing margins, is associated with prominent lymphocytic infiltrate and is 
cytokeratin 5 and 6,. and Her2/neu negative. which supports the diffarentiation. cytokeratin 14, cytokeratin 17, 
epidermal growth factor and vimentin positive and estrogen and progesterone receptor. CASE SYNOPSIS: SYNOPTIC - 
PRIMARY INVASIVE CARCINOMA OF BREAST. LATERALITY: Right. PROCEDURE: Segmental. Not specified. SIZE OF TUMOR: 
Maximum dimension invasive component: 1.4 cm. MULTICENTRICITY/MULTIFOCALITY OF INVASIVE FOCI: TUMOR TYPE (invasive 
component): Other Type(s): basal-like type. NOTTINGHAM SCORE: Nuclear grade: 3. Tubule formation: 3. Mitotic 
activity score: 3. Total Nottingham score: 9. Nottingham grade (1, 2, 3): 3. ANGIOLYMPHATIC INVASION: DERMAL 
LYMPHATIC INVASION: CALCIFICATION: TUMOR TYPE, IN SITU: Solid, Comedo. SURGICAL MARGINS INVOLVED BY INVASIVE 
COMPONENT: Distance of invasive tumor to closest margin: 0.3 mm. SURG MARGINS INVOLVED BY IN SITU COMPONENT: LYMPH 
NODES POSITIVE: 0. LYMPH NODES EXAMINED: 1. METHOD(S) OF LYMPH NODE EXAMINATION: H/E stain, Keratin stain. SENTINEL
NODE METASTASIS: SIZE OF NODAL METASTASES: Diameter of largest lymph node metastasis: 0 mm. LYMPH NODE 
METASTASIS(-ES) WITH EXTRACAPSULAR EXTENSION: NON-NEOPLASTIC BREAST TISSUE: FCD. T STAGE, PATHOLOGIC: N STAGE, 
PATHOLOGIC: pNO. pMX. M STAGE, PATHOLOGIC: ESTROGEN RECEPTORS: negative. PROGESTERONE RECEPTORS: negative. 
HER2/NEU: zero or 1+.

In [56]:
# pickle the list of examples
with open("tcga_examples.pkl", "wb") as f:
    pickle.dump(examples, f)
print(f"Saved {len(examples)} examples to tcga_examples.pkl")
# Load the examples back to verify
with open("tcga_examples.pkl", "rb") as f:
    loaded_examples = pickle.load(f)
print(f"Loaded {len(loaded_examples)} examples from tcga_examples.pkl")

Saved 9523 examples to tcga_examples.pkl
Loaded 9523 examples from tcga_examples.pkl
